# Modeling Game

In [21]:
%matplotlib inline
import numpy as np
import model_fitting as mf
import tellurium as te
import matplotlib.pyplot as plt
import pandas as pd
from collections import namedtuple
import os

In [32]:
# Symbols
# Column names
TIME = 'time'

In [3]:
# Initial model
model = '''
// Created by libAntimony v3.9.4
model *pathway()

  // Compartments and Species:
  species INPUT, P1, mRNA1, P2, mRNA2, P3, mRNA3, P4, mRNA4, P5, mRNA5, P6;
  species mRNA6, P7, mRNA7, P8, mRNA8;

  // Reactions:
  J1:  => mRNA1; L1 + Vm1*((K1_1*INPUT^H1 + K2_1*P4^H1 + K1_1*K3_1*INPUT^H1*P4^H1)/(1 + K1_1*INPUT^H1 + K2_1*P4^H1 + K1_1*K3_1*INPUT^H1*P4^H1)) - d_mRNA1*mRNA1;
  F1:  => P1; a_protein1*mRNA1 - d_protein1*P1;
  J2:  => mRNA2; L2 + Vm2*(K1_2*P4^H2/(1 + K1_2*P4^H2)) - d_mRNA2*mRNA2;
  F2:  => P2; a_protein2*mRNA2 - d_protein2*P2;
  J3:  => mRNA3; L3 + Vm3*(K1_3*P6^H3/(1 + K1_3*P6^H3)) - d_mRNA3*mRNA3;
  F3:  => P3; a_protein3*mRNA3 - d_protein3*P3;
  J4:  => mRNA4; L4 + Vm4*(1/(1 + K1_4*P2^H4)) - d_mRNA4*mRNA4;
  F4:  => P4; a_protein4*mRNA4 - d_protein4*P4;
  J5:  => mRNA5; L5 - d_mRNA5*mRNA5;
  F5:  => P5; a_protein5*mRNA5 - d_protein5*P5;
  J6:  => mRNA6; L6 + Vm6*(K1_6*P7^H6/(1 + K1_6*P7^H6 + K2_6*P1^H6 + K1_6*K2_6*P7^H6*P1^H6)) - d_mRNA6*mRNA6;
  F6:  => P6; a_protein6*mRNA6 - d_protein6*P6;
  J7:  => mRNA7; L7 - d_mRNA7*mRNA7;
  F7:  => P7; a_protein7*mRNA7 - d_protein7*P7;
  J8:  => mRNA8; L8 + Vm8*(1/(1 + K1_8*P1^H8)) - d_mRNA8*mRNA8;
  F8:  => P8; a_protein8*mRNA8 - d_protein8*P8;

  // Species initializations:
  INPUT = 1;
  P1 = 0;
  mRNA1 = 0;
  P2 = 0;
  mRNA2 = 0;
  P3 = 0;
  mRNA3 = 0;
  P4 = 0;
  mRNA4 = 0;
  P5 = 0;
  mRNA5 = 0;
  P6 = 0;
  mRNA6 = 0;
  P7 = 0;
  mRNA7 = 0;
P8 = 0;
  mRNA8 = 0;

  // Variable initializations:
  d_protein1 = 0.01576525;
  d_mRNA1 = 0.600013;
  L1 = 0.0284569399501349;
  Vm1 = 1.16380673480284;
  a_protein1 = 0.089105566248939;
  H1 = 4.52340391321994;
  K1_1 = 0.0269204907071558;
  K2_1 = 0.0169635567504703;
  K3_1 = 0.0114278645720656;
  d_protein2 = 0.0100753359178861;
  d_mRNA2 = 0.607263145327485;
  L2 = 0.01126375;
  Vm2 = 0.855433454948057;
  a_protein2 = 0.0825104648147814;
  H2 = 3.21939257313515;
  K1_2 = 0.0170170903653747;
  d_protein3 = 0.0165270958726424;
  d_mRNA3 = 1.4096553751623;
  L3 = 0.0108824170117358;
  Vm3 = 1.61383118795785;
  a_protein3 = 0.118672807163739;
  H3 = 4.57189341195625;
  K1_3 = 0.0133069236136431;
  d_protein4 = 0.0205716618573404;
  d_mRNA4 = 1.19069657063437;
  L4 = 0.0142166568749038;
  Vm4 = 0.916251896011744;
  a_protein4 = 0.0862964088164644;
  H4 = 5.00512303222327;
  K1_4 = 0.0179894288457716;
  d_protein5 = 0.0180685727313577;
  d_mRNA5 = 0.911653907722886;
  L5 = 0.0124824037745626;
  a_protein5 = 0.106293056264931;
  d_protein6 = 0.0178004316181647;
  d_mRNA6 = 0.67287496171115;
  L6 = 0.0166516087433626;
  Vm6 = 0.889302076744445;
  a_protein6 = 0.0890528700251159;
  H6 = 5.58112408673455;
  K1_6 = 0.0139445776013774;
  K2_6 = 0.0121764364668572;
  d_protein7 = 0.0206180615545929;
  d_mRNA7 = 0.618711430584466;
  L7 = 0.0178636133026099;
  a_protein7 = 0.0764169841455256;
  d_protein8 = 0.0131749080364666;
  d_mRNA8 = 1.17483328491068;
  L8 = 0.0165535374304033;
  Vm8 = 0.87881445337468;
  a_protein8 = 0.103749989801903;
  H8 = 2.17775388441324;
  K1_8 = 0.0168599518440462;

  // Other declarations:
  const d_protein1, d_mRNA1, L1, Vm1, a_protein1, H1, K1_1, K2_1, K3_1, d_protein2;
  const d_mRNA2, L2, Vm2, a_protein2, H2, K1_2, d_protein3, d_mRNA3;
  const L3, Vm3, a_protein3, H3, K1_3, d_protein4, d_mRNA4, L4;
  const Vm4, a_protein4, H4, K1_4, d_protein5, d_mRNA5, L5;
  const a_protein5, d_protein6, d_mRNA6, L6, Vm6, a_protein6;
  const H6, K1_6, K2_6, d_protein7, d_mRNA7, L7, a_protein7;
  const d_protein8, d_mRNA8, L8, Vm8, a_protein8, H8, K1_8;
end
'''
rr = te.loada(model)

In [61]:
def simulate(model, is_mrna=True, is_protein=True, is_input=False):
    rr = te.loada(model)
    data = rr.simulate(0, 1200, 120)
    length = np.shape(data)[0]
    df_data = pd.DataFrame(data)
    columns = [s[1:-1] for s in data.colnames]
    columns[0] = TIME
    times = [10*v for v in range(length)]
    df_data.columns = columns
    df_data[TIME] = times
    df_data = df_data.set_index(TIME)
    #
    def delColumn(string):
        for col in df_data.columns:
            if col[:len(string)] == string:
                del df_data[col]
    #
    if not is_mrna:
        delColumn("mRNA")
    if not is_protein:
        delColumn("P")
    if not is_input:
        delColumn("INPUT")
    return df_data

In [54]:
a = "hello"
a[:2]

'he'

In [64]:
df = simulate(model, is_protein=False, is_input=True)
df.head()

,INPUT,mRNA1,mRNA2,mRNA3,mRNA4,mRNA5,mRNA6,mRNA7,mRNA8
time,,,,,,,,,
0,1.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
10,1.0,0.099374,0.020919,0.00772,0.781444,0.013691,0.024719,0.028816,0.762087
20,1.0,0.130436,0.042435,0.00772,0.781449,0.013692,0.024747,0.028872,0.761940
30,1.0,0.245253,0.090163,0.00772,0.781449,0.013692,0.024747,0.028872,0.761423
40,1.0,0.444409,0.155441,0.00772,0.781447,0.013692,0.024747,0.028872,0.759376


In [39]:
def getRNASeq(csv_file):
    path = os.path.join("data", csv_file)
    df = pd.read_csv(path)
    df[TIME] = [int(v) for v in df[TIME]]
    df = df.set_index(TIME)
    return df

In [40]:
df = getRNASeq("Curie_Wild_0_RNASeqResult_Clean.csv")
df.head()

,mRNA1,mRNA2,mRNA3,mRNA4,mRNA5,mRNA6,mRNA7,mRNA8
time,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.099307,0.020844,0.007720,0.781444,0.013691,0.084857,2.404909,0.746052
20,0.129347,0.041853,0.007983,0.781449,0.014414,1.000481,2.361672,0.688322
30,0.241193,0.088704,0.055608,0.781449,0.065749,1.286064,2.248185,0.621902
40,0.436952,0.153115,0.357630,0.781445,0.282754,1.326094,2.102741,0.566512


In [31]:
df = simulate(model)
df

,INPUT,P1,mRNA1,P2,mRNA2,P3,mRNA3,P4,mRNA4,P5,mRNA5,P6,mRNA6,P7,mRNA7,P8,mRNA8
time,,,,,,,,,,,,,,,,,
0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10.084034,1.0,0.069080,0.099374,0.012770,0.020919,0.007953,0.007720,0.567295,0.781444,0.012059,0.013691,0.017535,0.024719,0.017099,0.028816,0.687045,0.762087
20.168067,1.0,0.150648,0.130436,0.034989,0.042435,0.015242,0.007720,1.075146,0.781449,0.023467,0.013692,0.034993,0.024747,0.033971,0.028872,1.348128,0.761940
30.252101,1.0,0.278973,0.245253,0.082831,0.090163,0.021412,0.007720,1.487855,0.781449,0.032975,0.013692,0.049586,0.024747,0.047682,0.028872,1.926677,0.761423
40.336134,1.0,0.522420,0.444409,0.171683,0.155441,0.026634,0.007720,1.823246,0.781447,0.040899,0.013692,0.061781,0.024747,0.058819,0.028872,2.432137,0.759376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159.663866,1.0,4.101532,0.725676,1.990259,0.243030,0.055441,0.007721,2.114355,0.504028,0.080547,0.013692,0.123805,0.024747,0.107010,0.028872,4.428001,0.562299
1169.747899,1.0,4.101532,0.725676,1.990259,0.243030,0.055441,0.007721,2.114355,0.504028,0.080547,0.013692,0.123805,0.024747,0.107010,0.028872,4.428001,0.562299
1179.831933,1.0,4.101533,0.725675,1.990259,0.243030,0.055441,0.007721,2.114355,0.504028,0.080547,0.013692,0.123805,0.024747,0.107010,0.028872,4.428002,0.562299


In [5]:
data = rr.simulate(0, 1200, 120)

In [9]:
df_data = pd.DataFrame(data)

In [11]:
data.colnames

['time',
 '[INPUT]',
 '[P1]',
 '[mRNA1]',
 '[P2]',
 '[mRNA2]',
 '[P3]',
 '[mRNA3]',
 '[P4]',
 '[mRNA4]',
 '[P5]',
 '[mRNA5]',
 '[P6]',
 '[mRNA6]',
 '[P7]',
 '[mRNA7]',
 '[P8]',
 '[mRNA8]']